# Prepare RDF for SKOSMOS

SKOSMOS has a simple, plain-SKOS [data model](https://github.com/NatLibFi/Skosmos/wiki/Data-Model). Our data model derives from this in two major ways that break SKOSMOS’ expectations:

1. We use SKOS-XL as the lexicalization scheme, and
2. we use the more specific sub-properties of `skos:broader` defined by skos-thes.

We map these to plain SKOS before loading the dataset into SKOSMOS.

In [1]:
import rdflib
from rdflib.namespace import SKOS, RDFS

skosxl = rdflib.Namespace('http://www.w3.org/2008/05/skos-xl#')
skos_thes = rdflib.Namespace('http://purl.org/iso25964/skos-thes#')

In [2]:
g = rdflib.Graph()
g.parse('diga_terms_vocbench.ttl')

<Graph identifier=N467240c86bd14262af7a09b6fbdd9a6a (<class 'rdflib.graph.Graph'>)>

## Broaders/Narrowers

We use `skos-thes:broaderGeneric` and `skos-thes:broaderPartitive`. SKOSMOS does not use these for building the hierarchy tree, so we have to make them known as subtypes of `skos:broader`. We also explicitly add the implicit inverse (narrower) properties.

In [3]:
qres = g.query(
    """SELECT ?part ?parent
       WHERE {
          ?part skos-thes:broaderPartitive ?parent .
       } LIMIT 3""")

for part, parent in qres:
    print(part, parent)

https://w3id.org/diga/terms/100088298 https://w3id.org/diga/terms/2994146436
https://w3id.org/diga/terms/1460785757 https://w3id.org/diga/terms/2994146436
https://w3id.org/diga/terms/3163172208 https://w3id.org/diga/terms/2994146436


Add sub-properties for skos-thes.

In [4]:
g.add((skos_thes.broaderPartitive, RDFS.subPropertyOf, SKOS.broader))
g.add((skos_thes.broaderGeneric, RDFS.subPropertyOf, SKOS.broader))
g.add((skos_thes.narrowerPartitive, RDFS.subPropertyOf, SKOS.narrower))
g.add((skos_thes.narrowerGeneric, RDFS.subPropertyOf, SKOS.narrower))

Add inverse relations.

In [5]:
g.update("""
INSERT { 
    ?parent skos-thes:narrowerPartitive ?part .
}
WHERE {
    ?part skos-thes:broaderPartitive ?parent .
}
""")

In [6]:
g.update("""
INSERT { 
    ?parent skos-thes:narrowerGeneric ?part .
}
WHERE {
    ?part skos-thes:broaderGeneric ?parent .
}
""")

We also have a few left-over plain `skos:broaders`. For the time being, we also add the reverse relations for these.

In [7]:
g.update(
    """INSERT { ?parent skos:narrower ?facet . }
       WHERE {
          ?facet skos:broader ?parent .
       }""")

## Lexicalization

We add plain skos labels based on the SKOS-XL properties.

In [8]:
qres = g.query(
    """SELECT ?concept ?label
       WHERE {
           ?concept skosxl:prefLabel ?xlabel .
           ?xlabel skosxl:literalForm ?label .
       } LIMIT 10""")

for c, l in qres:
    print(c, l)

https://w3id.org/diga/terms/1000747715 volti in alto e in basso, a foglie lanceolate e boccioli di fiori
https://w3id.org/diga/terms/1000747715 spreading upwards and downwards, with lanceolate leaves and blossoms
https://w3id.org/diga/terms/100088298 orlo
https://w3id.org/diga/terms/100088298 edge
https://w3id.org/diga/terms/1001583333 semiarchetto laterale (dx./ sn.)
https://w3id.org/diga/terms/1001583333 side small half arch (right/ left)
https://w3id.org/diga/terms/1001756703 torus
https://w3id.org/diga/terms/1001756703 toro
https://w3id.org/diga/terms/100479688 piramide a gradini
https://w3id.org/diga/terms/100479688 stepped pyramid


In [9]:
g.update(
    """INSERT { ?concept skos:prefLabel ?label . }
       WHERE {
           ?concept skosxl:prefLabel ?xlabel .
           ?xlabel skosxl:literalForm ?label .
       }""")

In [10]:
g.update(
    """INSERT { ?concept skos:altLabel ?label . }
       WHERE {
           ?concept skosxl:altLabel ?xlabel .
           ?xlabel skosxl:literalForm ?label .
       }""")

## Editorial notes

We remove the editorial notes that are not meant for public display.

In [11]:
g.update(
    """DELETE { ?concept skos:editorialNote ?note }
    WHERE {
        ?concept skos:editorialNote ?note .
        FILTER ( STRSTARTS ( ?note, "Figure label" ) )
    }""")

## Save

In [12]:
with open('diga_terms_skosmos.ttl', 'wb') as outfile:
    g.serialize(destination=outfile, format='turtle')